In [2]:
'''
# Inspect Image Annotation Data created by VIA tool
Inspect and visualize data loading and pre-processing code.
'''
import os
import sys
import random
import math
import collections
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library


from importlib import import_module
import ../../external/Mask_RCNN/samples/ods
from ods import get_dnncfg
from ods import ODS
import core.utils as odsutils


%matplotlib inline

SyntaxError: invalid syntax (<ipython-input-2-07c49060508b>, line 22)

In [2]:
'''
## Load YAML file as easy dictionary object
'''
def yaml_load(fileName):
  import yaml
  from easydict import EasyDict as edict
  fc = None
  with open(fileName, 'r') as f:
    # fc = edict(yaml.load(f))
    fc = edict(yaml.safe_load(f))

  return fc


def get_cfgmap(cfgfile, datacfg):
  cfgmap = {
      'source':datacfg.NAME
      ,'cfgfile':cfgfile
      ,'datacfg':datacfg
    }
  return cfgmap


def load_dataset_cfg(cfgfiles):
  '''
  ## load_dataset_cfg
  '''
  ccfg = []
  for cfgfile in cfgfiles:
    datacfg = odsutils.yaml_load(cfgfile)
    cfgmap = get_cfgmap(cfgfile, datacfg)
    ccfg.append(cfgmap)

  return ccfg


def get_dataset_dnncfg(datacfg, subset = "train"):
    '''
    ## Load configurations file and create config object
    '''
    from importlib import import_module
    from core.InferenceConfig import InferenceConfig
    
    print("datacfg.DATACLASS: {}".format(datacfg.DATACLASS))

    mod = import_module(datacfg.DATACLASS)
    modcls = getattr(mod, datacfg.DATACLASS)
    
    dataset = modcls(datacfg.NAME)
    dataset.load_data(subset, datacfg)
    dataset.prepare()

    class_names = [ i['name'] for i in dataset.class_info]
    num_classes = len(dataset.class_info)
    datacfg.CLASSES = class_names
    datacfg.CONFIG.NAME = dataset.name
    datacfg.CONFIG.NUM_CLASSES = num_classes

    dnncfg = InferenceConfig(datacfg.CONFIG)
    
    return dataset, dnncfg


def get_dataset_names(ccfg):
  ds_names = [i['source'] for i in ccfg]
  return ds_names


def get_dataset_cfg(ccfg, name=None):
  print("ods:get_dataset_cfg---------------------------->")

  ds_cfgs = []
  
  # if not ccfg or len(ccfg) < 1:
  #   return ds_cfgs

  if name:
    ds_names = get_dataset_names(ccfg)
    ds_index = [i for i, j in enumerate(ds_names) if j == name]
    print("ds_index:{}".format(ds_index))
    if len(ds_index) > 0:
      # ds_cfgs = [ccfg[l]['datacfg'] for l in ds_index]
      ds_cfgs = {ccfg[k]['source']:ccfg[k]['datacfg'] for k in ds_index}
  else:
    ds_cfgs = {l['source']:l['datacfg'] for k,l in enumerate(ccfg)}

  return ds_cfgs


In [3]:
'''
TBD:
* Images Stats:
    - total
    - dimensions: max, min, mean
* Annotations Stats:
    - total
    - area
'''

'''
## Configurations
Configurations are defined in `.yml` file
'''
'''
## Configurations
Configurations are defined in `.yml` file
'''
cfgroot = 'cfg'
cfgroot = 'cfg-dev'
cfgroot = 'cfg-local'
cfgroot = '/home/nikhil/Documents/ai-ml-dl/external/Mask_RCNN/samples/ods/cfg-local'

cfgfilename = 'hmd-cfg.yml'
cfgfilename = 'tsd-cfg.yml'
cfgfilename = 'road-cfg.yml'

cfgfilename = 'crosstrain-cfg-120319-1.yml'
cfgfilename = 'balloon-cfg.yml'
cfgfilename = 'road-asphalt_both_side-cfg.yml'
cfgfilename = 'coco_things-cfg.yml'

cfgfile = os.path.join(cfgroot, cfgfilename)
print("cfgfile:{}".format(cfgfile))

datacfg = yaml_load(cfgfile)
print(datacfg)
ccfg = None
error = False
dataset_cfg = None

## TBD: raise error
if 'DATASET_CFG' in datacfg:
  dataset_cfg = datacfg.DATASET_CFG
if dataset_cfg:
  ccfg = load_dataset_cfg(dataset_cfg)
else:
  error = True
  print("Empty DATASET_CFG! Please fix the configuration file: {}".format(cfgfile))

cifilepath = os.path.join(odsutils.getBasePath(cfgfile),'classinfo-'+os.path.basename(cfgfile)+'.json')

print("datacfg: {}".format(datacfg))
print("ccfg: {}".format(ccfg))

cfgfile:/home/nikhil/Documents/ai-ml-dl/external/Mask_RCNN/samples/ods/cfg-local/coco_things-cfg.yml
{'CLASS_IDS': None, 'RETURN_COCO': False, 'DATA_READ_THRESHOLD': -1, 'ANNOTATIONS': {'TRAIN': '/home/nikhil/Documents/ai-ml-dl-data/data/ms-coco/annotations/instances_train2014.json', 'VAL': '/home/nikhil/Documents/ai-ml-dl-data/data/ms-coco/annotations/instances_val2014.json'}, 'SCHEDULES': [{'LEARNING_RATE': 0.001, 'EPOCHS': 40, 'LAYERS': 'heads'}, {'LEARNING_RATE': 0.001, 'EPOCHS': 120, 'LAYERS': '4+'}, {'LEARNING_RATE': 0.0001, 'EPOCHS': 160, 'LAYERS': 'all'}], 'TRAIN_MODE': 'training', 'DATACLASS': 'HmdDataset', 'RETURN_HMD': None, 'WEIGHTS_PATH': '/home/nikhil/Documents/ai-ml-dl-data/data/matterport/object_detection_segmentation/1/mask_rcnn/weights/mask_rcnn_coco.h5', 'LOG_DIR': '/home/nikhil/Documents/ai-ml-dl-data/logs/mask_rcnn', 'CLASSES': ['person', 'bicycle', 'car', 'motorcycle', 'airplane,', 'bus', 'train', 'truck', 'boat', 'traffic light,', 'fire hydrant', 'stop sign', 'pa

NameError: name 'odsutils' is not defined